## 📦 Imports

In [1]:
import base64
import hmac
import json
import time
from urllib.parse import urlencode

import requests

## ⚙️ Fonctions


In [2]:
class BitgetClient:
    """Client Bitget pour les opérations Spot et Earn."""
    
    def __init__(self, api_key: str, api_secret: str, passphrase: str, api_host: str = "https://api.bitget.com"):
        """
        Initialise le client Bitget avec les credentials API.
        
        Paramètres
        ----------
        api_key : str
            Clé API Bitget
        api_secret : str
            Secret API Bitget  
        passphrase : str
            Passphrase API Bitget
        api_host : str, optionnel
            URL de base de l'API Bitget
        """
        self.api_key = api_key
        self.api_secret = api_secret
        self.passphrase = passphrase
        self.api_host = api_host
    
    def _timestamp_ms(self) -> str:
        """Timestamp actuel en millisecondes (format string selon spec Bitget)."""
        return str(int(time.time() * 1000))
    
    def _sign(self, ts: str, method: str, path: str, query: str = "", body: str = "") -> str:
        """
        Retourne la signature HMAC-SHA256 encodée en Base64 pour la requête.
        """
        prehash = f"{ts}{method.upper()}{path}{query}{body}"
        digest = hmac.new(self.api_secret.encode(), prehash.encode(), digestmod="sha256").digest()
        return base64.b64encode(digest).decode()
    
    def _headers(self, ts: str, signature: str) -> dict:
        """En-têtes HTTP requis par Bitget."""
        return {
            "ACCESS-KEY": self.api_key,
            "ACCESS-SIGN": signature,
            "ACCESS-PASSPHRASE": self.passphrase,
            "ACCESS-TIMESTAMP": ts,
            "Content-Type": "application/json",
            "locale": "en-US",
        }
    
    def _request(self, method: str, path: str, *, params: dict | None = None, body: dict | None = None):
        """
        Requête REST générique Bitget avec signature et gestion d'erreurs de base.
        Retourne le champ `data` de la réponse JSON.
        """
        params = params or {}
        body = body or {}
        query = "?" + urlencode(sorted(params.items())) if params else ""
        body_s = json.dumps(body) if body else ""

        ts = self._timestamp_ms()
        sign = self._sign(ts, method, path, query, body_s)
        url = f"{self.api_host}{path}{query}"

        r = requests.request(method, url, headers=self._headers(ts, sign), data=body_s, timeout=10)
        r.raise_for_status()

        payload = r.json()
        if payload.get("code") != "00000":
            raise RuntimeError(f"Erreur API Bitget: {payload}")

        return payload["data"]
    
    def _current_usdt_flexible_product_id(self) -> str:
        """
        Retourne le productId d'un produit d'épargne flexible USDT *actif*.
        Lève une exception si aucun n'est disponible.
        """
        products = self._request(
            "GET",
            "/api/v2/earn/savings/product",
            params={"coin": "USDT", "filter": "available_and_held"}
        )
        for p in products:
            if p["periodType"] == "flexible" and p["status"] == "in_progress":
                return p["productId"]

        raise ValueError("Aucun produit d'épargne flexible USDT actif trouvé (épuisé ?)")
    
    def subscribe_savings_usdt(self, amount_usdt: str) -> str:
        """
        Dépose `amount_usdt` (string, jusqu'à 8 décimales) dans l'épargne flexible USDT.
        Retourne l'orderId résultant.
        """
        pid = self._current_usdt_flexible_product_id()
        result = self._request(
            "POST",
            "/api/v2/earn/savings/subscribe",
            body={
                "periodType": "flexible",
                "productId": pid,
                "amount": amount_usdt
            }
        )
        return result["orderId"]
    
    def redeem_savings_usdt(self, amount_usdt: str) -> str:
        """
        Retire `amount_usdt` d'USDT de l'épargne flexible vers le Spot.
        Bitget impose au moins 1 minute entre les retraits du même produit.
        Retourne l'orderId résultant.
        """
        pid = self._current_usdt_flexible_product_id()
        result = self._request(
            "POST",
            "/api/v2/earn/savings/redeem",
            body={
                "periodType": "flexible",
                "productId": pid,
                "amount": amount_usdt
            }
        )
        return result["orderId"]
    
    def get_spot_balance(self, coin: str = "USDT", *, asset_type: str = "hold_only") -> dict:
        """
        Retourne le solde de votre portefeuille Spot pour `coin`.

        Paramètres
        ----------
        coin : str
            Symbole du token, ex. "USDT", "BTC". Par défaut "USDT".
        asset_type : str
            "hold_only" (défaut) ou "all". Correspond au paramètre `assetType` de Bitget.

        Retour
        ------
        dict
            Exemple:
            {
              "coin": "USDT",
              "available": "37.221645",
              "frozen": "0",
              "locked": "0",
              "limitAvailable": "0",
              "uTime": "1718081856000"
            }
        """
        data = self._request(
            "GET",
            "/api/v2/spot/account/assets",
            params={"coin": coin.upper(), "assetType": asset_type}
        )
        return data[0] if data else {}
    
    def _get_savings_assets(
        self,
        period_type: str,
        *,
        start_time: str | None = None,
        end_time: str | None = None,
        limit: str | None = None,
        id_less_than: str | None = None
    ) -> dict:
        """
        Récupère les actifs d'épargne selon les critères spécifiés.

        Paramètres
        ----------
        period_type : str
            Type de période:
            - "flexible" : dépôt à vue
            - "fixed" : dépôt à terme
        start_time : str, optionnel
            Timestamp de début en millisecondes (ex: "1597026383085")
            Par défaut: il y a 3 mois
        end_time : str, optionnel
            Timestamp de fin en millisecondes (ex: "1597026383085")
            Par défaut: maintenant
        limit : str, optionnel
            Nombre d'éléments par page. Par défaut: "20", maximum: "100"
        id_less_than : str, optionnel
            ID de pagination pour requêter les données plus anciennes
        """
        params = {"periodType": period_type}
        
        if start_time is not None:
            params["startTime"] = start_time
        if end_time is not None:
            params["endTime"] = end_time
        if limit is not None:
            params["limit"] = limit
        if id_less_than is not None:
            params["idLessThan"] = id_less_than

        return self._request(
            "GET",
            "/api/v2/earn/savings/assets",
            params=params
        )
    
    def get_flexible_savings_amount(self, coin: str = "USDT") -> str:
        """
        Retourne le montant détenu en épargne flexible pour une crypto donnée.

        Paramètres
        ----------
        coin : str
            Symbole du token, ex. "USDT", "BTC". Par défaut "USDT".

        Retour
        ------
        str
            Montant détenu. "0" si aucun actif trouvé pour cette crypto.
        """
        data = self._get_savings_assets("flexible")
        
        for asset in data.get("resultList", []):
            if asset.get("productCoin", "").upper() == coin.upper():
                return asset.get("holdAmount", "0")
        
        return "0"

## 🔐 Clef Api

In [3]:
API_KEY      = ""
API_SECRET   = ""
PASSPHRASE   = ""

client = BitgetClient(API_KEY, API_SECRET, PASSPHRASE)

## 🔍 Test de connexion


In [ ]:
try:
    balance = client.get_spot_balance("USDT")
    
    if balance:
        print("🎉 Connexion API réussie !")
        print(f"💰 Solde USDT disponible : {balance['available']} USDT")
        print(f"🔒 USDT gelés : {balance['frozen']} USDT")
        print(f"🔐 USDT verrouillés : {balance['locked']} USDT")
    else:
        print("⚠️  Aucun solde USDT trouvé dans le portefeuille Spot")
        
except Exception as e:
    print(f"❌ Erreur lors de la récupération du solde : {e}")
    print("🔧 Vérifiez vos clés API")


## 💰 Dépôt en épargne

In [ ]:
MONTANT_DEPOT = "10"

try:
    print(f"💰 Solde USDT Spot avant dépôt : {client.get_spot_balance('USDT')['available']} USDT")
    print(f"💰 Solde USDT épargne avant dépôt : {client.get_flexible_savings_amount('USDT')} USDT")
    print(f"🏦 Dépôt de {MONTANT_DEPOT} USDT dans l'épargne flexible...")
    
    client.subscribe_savings_usdt(MONTANT_DEPOT)
    
    print(f"✅ Dépôt réussi !")
    
    time.sleep(5)
    nouveau_solde_spot = client.get_spot_balance('USDT')['available']
    nouveau_solde_earn = client.get_flexible_savings_amount('USDT')
    
    print(f"💰 Nouveau solde USDT disponible en Spot : {nouveau_solde_spot} USDT")
    print(f"💰 Nouveau solde USDT épargné en Earn : {nouveau_solde_earn} USDT")
    
except Exception as e:
    print(f"❌ Erreur lors du dépôt : {e}")
    print("💡 Assurez-vous d'avoir suffisamment d'USDT dans votre portefeuille Spot")


## 💸 Retrait de l'épargne


In [ ]:
MONTANT_RETRAIT = "10"

try:
    print(f"💰 Solde USDT Spot avant retrait : {client.get_spot_balance('USDT')['available']} USDT")
    print(f"💰 Solde USDT épargne avant retrait : {client.get_flexible_savings_amount('USDT')} USDT")

    
    print(f"🏦 Retrait de {MONTANT_RETRAIT} USDT de l'épargne flexible...")
    
    order_id = client.redeem_savings_usdt(MONTANT_RETRAIT)
    
    print(f"✅ Retrait réussi !")
    
    time.sleep(5)
    nouveau_solde_spot = client.get_spot_balance('USDT')['available']
    nouveau_solde_earn = client.get_flexible_savings_amount('USDT')
    
    print(f"💰 Nouveau solde USDT disponible en Spot : {nouveau_solde_spot} USDT")
    print(f"💰 Nouveau solde USDT épargné en Earn : {nouveau_solde_earn} USDT")
    
except Exception as e:
    print(f"❌ Erreur lors du retrait : {e}")
    print("💡 Vérifiez que :")
    print("   - Vous avez des fonds dans l'épargne flexible")
    print("   - Au moins quelques secondes se sont écoulées depuis le dernier retrait")
    print("   - Le montant ne dépasse pas votre solde d'épargne")
